# DataFrame

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#pyspark-sql-module

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame_1') \
    .master('local[*]') \
    .getOrCreate()

In [ ]:
sc = spark.sparkContext

In [ ]:
data_path = '.../'

# Wczytanie DataFrame bezposrednio z JSON oraz CSV
people = spark.read.json(data_path+'people.json')
employees = spark.read.json(data_path+'employees.json')
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')

In [ ]:
people_txt.show()

****

### DataFrame to lista wierszy

In [ ]:
newPerson1 = Row(name='Greg', age=32)

In [ ]:
newPerson1

In [ ]:
newPerson1.name

In [ ]:
newPerson1.age

In [ ]:
newPerson1['age']

In [ ]:
'age' in newPerson1

In [ ]:
newPerson = Row("age", "name")

In [ ]:
newPerson2 = newPerson(24, 'Alice')

In [ ]:
newPerson2

In [ ]:
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [ ]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, newPerson5, newPerson6, newPerson7])

In [ ]:
newPeopleDF.show()

### Przechodzenie RDD <-> DF

In [ ]:
type(people)

In [ ]:
people

In [ ]:
people.collect()

DF -> RDD 

In [ ]:
type(people.rdd)

In [ ]:
people.rdd.collect()

In [ ]:
people.rdd.map(tuple).collect()

RDD -> DF

In [ ]:
people.rdd.toDF()

In [ ]:
people.rdd.map(tuple).toDF().collect()

Podanie schematu wprost przy tworzeniu DF

Typy danych: http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#module-pyspark.sql.types <br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#data-types <br>
Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])
df = spark.createDataFrame([[1,2],[3,4]], schema)

In [ ]:
# Data Frame nadal jest lista wierszy
df.show()

> **TODO**: Z podanego RDD utworz DataFrame. <br>
Wskazowka: DataFrame stworzymy latwo z RDD Rows<br>

#### Ogolne wiadomosci na temat danych
printSchema, show, columns, dtypes <br>
Znane z RDD np: count, take, head

In [ ]:
people.show()

In [ ]:
people.printSchema() ## Print the schema in a tree format

In [ ]:
employees.printSchema()

In [ ]:
people_txt.printSchema()

In [ ]:
people_txt.show(1)

In [ ]:
people.show(1)

In [ ]:
people_txt.show(1)

Liczba wierszy

In [ ]:
people.count()

Lista kolumn

In [ ]:
people.columns

Lista kolumn wraz z typami danych

In [ ]:
people.dtypes

#### Odwolania do poszczegolnych kolumn

In [ ]:
people.age

In [ ]:
people['age']

In [ ]:
people[0]

#### Dodanie/usuniecie kolumny
withColumn, drop

In [ ]:
people.show(2)

In [ ]:
people.withColumn(colName = 'ageNextYear', col = people.age +1).show(2)

In [ ]:
people.drop('age').show()

#### Zmiany nazwy kolumny

DataFrame people_txt ma domyslne nazwy kolumn ze Sparka. Niewiele nam one mowia.

In [ ]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')
people_txt.show(1)

#### Podstawowe statystyki kolumn w DataFrames
describe

In [ ]:
people.describe()

In [ ]:
employees.describe('salary').show()

#### Braki danych
isNull, isNotNull<br>
fillna, dropna, replace

In [ ]:
newPeopleDF.show()

In [ ]:
newPeopleDF.filter(newPeopleDF.age.isNull()).show()

In [ ]:
newPeopleDF.filter(newPeopleDF.age.isNotNull()).show()

In [ ]:
newPeopleDF.fillna(-1).show()

In [ ]:
newPeopleDF.fillna({'age':-1, 'name':'unknown'}).show()

In [ ]:
newPeopleDF.fillna({'name':'unknown'}).replace('unknown', 'NN').show()

In [ ]:
newPeopleDF.dropna().show()

In [ ]:
newPeopleDF.dropna(subset='age').show()

In [ ]:
newPeopleDF.dropna?

#### Funkcje wprost ze skladni SQL 
select, where (wymienne z filter), orderBy

In [ ]:
people.show()

In [ ]:
people.\
select('name').\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()

> **TODO**: Wyswietl imiona ludzi ze zbioru 'people' starszych niz 29 lat

#### Operacje na zbiorach
union - dziala jak UNION ALL w SQL. <br>
intersect (INTERSECT z SQLa), subtract (EXCEPT z SQLa)

In [ ]:
people.union?

In [ ]:
people.show()

In [ ]:
people_txt.show()

In [ ]:
newPeopleDF.show()